In [1]:
import geopandas as gpd 
from shapely.geometry import LineString, Point, box
from tqdm.auto import tqdm
import math

In [2]:
lines = gpd.read_file('Data/TorrentBay_rates.shp')
def get_azimuth(line):
    p1, p2 = line.coords[1:]
    azimuth = math.degrees(math.atan2(p2[0]-p1[0], p2[1]-p1[1]))
    if azimuth < 0:
        azimuth += 360
    return azimuth
lines["Azimuth"] = lines.geometry.apply(get_azimuth)
lines["length"] = lines.length

In [3]:
MHWL = gpd.read_file('Data/nz-MHWL.shp')
MHWL = MHWL[MHWL.intersects(box(*lines.total_bounds))]

In [4]:
lines["intersect_point"] = lines.intersection(MHWL.unary_union)
lines

,ObjectID_1,BaselineID,TransOrder,TransEdit,transectId,ShrCount,LRR,LR2,LCI,LSE,...,SCE,WLR,WR2,WCI,WSE,TCD,geometry,Azimuth,length,intersect_point
0,1.0,0.0,1.0,0.0,1.0,10.0,0.02,0.04,0.05,2.29,...,7.11,0.00,0.00,0.05,0.67,0.00,"LINESTRING Z (1605478.052 5466338.300 0.000, 1...",239.162167,90.002,POINT Z (1605454.653 5466324.330 0.000)
1,2.0,0.0,2.0,0.0,2.0,10.0,0.03,0.12,0.04,2.07,...,6.88,0.01,0.03,0.05,0.63,10.00,"LINESTRING Z (1605472.926 5466346.886 0.000, 1...",239.162167,90.002,POINT Z (1605446.397 5466331.047 0.000)
2,3.0,0.0,3.0,0.0,3.0,10.0,0.02,0.14,0.04,1.87,...,6.24,0.01,0.04,0.05,0.60,20.00,"LINESTRING Z (1605467.800 5466355.472 0.000, 1...",239.162167,90.002,POINT Z (1605439.026 5466338.293 0.000)
3,4.0,0.0,4.0,0.0,4.0,10.0,0.02,0.15,0.04,1.78,...,5.27,0.02,0.06,0.05,0.61,30.00,"LINESTRING Z (1605462.673 5466364.058 0.000, 1...",239.162169,90.002,POINT Z (1605432.715 5466346.172 0.000)
4,5.0,0.0,5.0,0.0,5.0,10.0,0.02,0.14,0.04,1.75,...,5.28,0.02,0.07,0.04,0.56,40.00,"LINESTRING Z (1605457.547 5466372.644 0.000, 1...",239.162168,90.002,POINT Z (1605428.487 5466355.295 0.000)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,466.0,12.0,466.0,0.0,466.0,8.0,-0.21,0.65,0.12,3.36,...,17.03,-0.21,0.55,0.15,1.10,12140.35,"LINESTRING Z (1604435.973 5474504.870 0.000, 1...",285.851929,90.002,LINESTRING Z EMPTY
463,467.0,12.0,467.0,0.0,467.0,8.0,-0.19,0.56,0.13,3.56,...,16.46,-0.19,0.43,0.17,1.23,12150.35,"LINESTRING Z (1604438.704 5474514.490 0.000, 1...",285.851927,90.002,LINESTRING Z EMPTY
464,468.0,12.0,468.0,0.0,468.0,8.0,-0.15,0.54,0.11,3.00,...,12.82,-0.15,0.44,0.13,0.97,12160.35,"LINESTRING Z (1604441.654 5474524.045 0.000, 1...",287.241459,90.002,LINESTRING Z EMPTY
465,469.0,12.0,469.0,0.0,469.0,8.0,0.02,0.03,0.09,2.40,...,6.11,0.02,0.02,0.13,0.92,12170.35,"LINESTRING Z (1604447.621 5474531.425 0.000, 1...",324.462323,90.002,LINESTRING Z EMPTY


In [5]:
def map():
    m = MHWL.explore()
    lines["intersects"] = ~lines.intersect_point.is_empty
    lines.explore("intersects", cmap=["red", "green"], legend=True, m=m)
    lines.intersect_point.explore(m=m, color="magenta")
    display(m)
map()

In [6]:
sum(lines.intersect_point.is_empty)

60

In [7]:
def calculate_new_coordinates(old_x, old_y, bearing, distance):
    bearing_radians = math.radians(bearing)
    new_x = old_x + (distance * math.sin(bearing_radians))
    new_y = old_y + (distance * math.cos(bearing_radians))
    point = Point(new_x, new_y)
    assert not point.is_empty
    return point

def extend(line, distance=-90):
    p1, p2 = line.coords[1:]
    azimuth = get_azimuth(line)
    new_point = calculate_new_coordinates(p1[0], p1[1], azimuth, distance)
    return LineString([new_point, Point(p2[0], p2[1])])

lines.geometry.update(lines[lines.intersect_point.is_empty].geometry.apply(extend))
lines["length"] = lines.length
lines["intersect_point"] = lines.intersection(MHWL.unary_union)
lines

/tmp/ipykernel_3397/1002817677.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  lines.geometry.update(lines[lines.intersect_point.is_empty].geometry.apply(extend))


,ObjectID_1,BaselineID,TransOrder,TransEdit,transectId,ShrCount,LRR,LR2,LCI,LSE,...,WLR,WR2,WCI,WSE,TCD,geometry,Azimuth,length,intersect_point,intersects
0,1.0,0.0,1.0,0.0,1.0,10.0,0.02,0.04,0.05,2.29,...,0.00,0.00,0.05,0.67,0.00,"LINESTRING Z (1605478.052 5466338.300 0.000, 1...",239.162167,90.002,POINT Z (1605454.653 5466324.330 0.000),True
1,2.0,0.0,2.0,0.0,2.0,10.0,0.03,0.12,0.04,2.07,...,0.01,0.03,0.05,0.63,10.00,"LINESTRING Z (1605472.926 5466346.886 0.000, 1...",239.162167,90.002,POINT Z (1605446.397 5466331.047 0.000),True
2,3.0,0.0,3.0,0.0,3.0,10.0,0.02,0.14,0.04,1.87,...,0.01,0.04,0.05,0.60,20.00,"LINESTRING Z (1605467.800 5466355.472 0.000, 1...",239.162167,90.002,POINT Z (1605439.026 5466338.293 0.000),True
3,4.0,0.0,4.0,0.0,4.0,10.0,0.02,0.15,0.04,1.78,...,0.02,0.06,0.05,0.61,30.00,"LINESTRING Z (1605462.673 5466364.058 0.000, 1...",239.162169,90.002,POINT Z (1605432.715 5466346.172 0.000),True
4,5.0,0.0,5.0,0.0,5.0,10.0,0.02,0.14,0.04,1.75,...,0.02,0.07,0.04,0.56,40.00,"LINESTRING Z (1605457.547 5466372.644 0.000, 1...",239.162168,90.002,POINT Z (1605428.487 5466355.295 0.000),True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,466.0,12.0,466.0,0.0,466.0,8.0,-0.21,0.65,0.12,3.36,...,-0.21,0.55,0.15,1.10,12140.35,"LINESTRING (1604522.551 5474480.286, 1604349.3...",285.851929,180.001,POINT (1604456.940 5474498.916),False
463,467.0,12.0,467.0,0.0,467.0,8.0,-0.19,0.56,0.13,3.56,...,-0.19,0.43,0.17,1.23,12150.35,"LINESTRING (1604525.283 5474489.906, 1604352.1...",285.851927,180.001,POINT (1604458.651 5474508.826),False
464,468.0,12.0,468.0,0.0,468.0,8.0,-0.15,0.54,0.11,3.00,...,-0.15,0.44,0.13,0.97,12160.35,"LINESTRING (1604527.610 5474497.369, 1604355.6...",287.241459,180.001,"MULTIPOINT (1604461.228 5474517.970, 1604488.8...",False
465,469.0,12.0,469.0,0.0,469.0,8.0,0.02,0.03,0.09,2.40,...,0.02,0.02,0.13,0.92,12170.35,"LINESTRING (1604499.934 5474458.189, 1604395.3...",324.462323,180.001,POINT (1604460.097 5474513.960),False


In [8]:
lines.intersect_point.geom_type.value_counts()

Point         413
MultiPoint     54
Name: count, dtype: int64

In [9]:
map()

In [10]:
assert sum(lines.intersect_point.is_empty) == 0

In [11]:
lines.drop(columns="intersect_point").to_file('Data/extended_transects.geojson')